## Simulating Economic Impact Using Leontief I-O Model

In [1]:
import numpy as np
import pandas as pd
import pathlib

### Read in I-O Table Data

In [2]:
# _1__ Import data from excel file

path = pathlib.Path(r"C:\Users\halen\Downloads\py-leontief_input_data.xlsx")

df = pd.read_excel(path)
df = df.fillna(0.e-20)

df.head()

,Industry,A_i,B_i,C_i,Final Demand,Total Output
0,A_i,20,60,30,90.0,200.0
1,B_i,80,90,20,110.0,300.0
2,C_i,40,30,10,20.0,100.0
3,Value Add,60,120,40,0.0,220.0
4,Total,220,285,110,225.0,0.0


### Updated Row Totals 

In [3]:
new_total = df.iloc[-1, 1:]

new_total.head()

A_i               220
B_i               285
C_i               110
Final Demand    225.0
Total Output      0.0
Name: 4, dtype: object

### RAS Matrix Rebalance

In [4]:
# Create initial Direct Requirements Matrix

df_coefficients = df.iloc[:-1, 1:-1]

cols = df_coefficients.columns

coefficients_array = np.array(df_coefficients)

df_coefficients.head()

,A_i,B_i,C_i,Final Demand
0,20,60,30,90.0
1,80,90,20,110.0
2,40,30,10,20.0
3,60,120,40,0.0


In [5]:
# Set variables and other requisites

calc_rows_total = np.array([])
row_conversion_factor = np.array([])
col_conversion_factor = np.array([])

rows_total = coefficients_array.shape[0]
cols_total = coefficients_array.shape[1]

max_diff = 0
iter_count = 0
iter_limit = 1000

In [6]:
# Calculate row conversion factors

for idx in range(rows_total):
    row1 = np.sum(coefficients_array[idx])
    row2 = new_total[idx]

    if row1 != row2:
        p = row2 / row1
    else:
        p = 1

    row_conversion_factor = np.append(row_conversion_factor, p)

row_total_sum = (np.sum(row_conversion_factor) - rows_total) / rows_total

print(f"New row adjustment factors indicate a {row_total_sum:.6f}% difference")

New row adjustment factors indicate a 0.043182% difference


In [7]:
# Rebalancing Sequence

new_array = np.array(coefficients_array)
iter_stats = {}
closest_match = row_total_sum
accuracy_improving = True


# If row_total_sum indicates initial misalignment, run RAS rebalancing
if abs(row_total_sum) > 0.0005:

    # Stop when current iter better than previous
    while accuracy_improving:

        iter_count += 1
        # Calculate row conversion factors
        for idx in range(rows_total):

            row1 = np.sum(new_array[idx])
            row2 = new_total[idx]

            if row1 != row2:
                p = row2 / row1
            else:
                p = 1

            row_conversion_factor = np.append(row_conversion_factor, p)

        row_total_sum = (np.sum(row_conversion_factor) - rows_total) / rows_total


        # Adjust i,j by col_conversion_factor
        new_rows = []

        for row in enumerate(new_array):
            row_ref = row[0]

            for col in enumerate(new_array):
                col_ref = col[0]

                row1_1 = new_array[row_ref, col_ref] * row_conversion_factor[row_ref]

                new_rows.append(row1_1)

                col_ref = col_ref + 1

            row_ref = row_ref + 1

        new_array = np.array(new_rows).reshape(rows_total, cols_total)

        for idx in range(rows_total):
            calc_rows_total = np.append(calc_rows_total, np.sum(new_array[idx]))
            idx = idx + 1

        # Calculate column conversion factors
        for idx in range(cols_total):

            array = new_array.sum(axis=0)

            col1 = array[idx]

            col2 = new_total[idx]

            if col1 != col2:
                p = col2 / col1

            else:
                p = 1

            col_conversion_factor = np.append(col_conversion_factor, p)

        col_total_sum = (np.sum(col_conversion_factor) - cols_total) / cols_total

        # Adjust i,j by row_conversion_factor
        new_cols = []

        for col in enumerate(new_array):
            col_ref = col[0]
            col_array = new_array[:,col_ref]

            for row in enumerate(new_array):
                row_ref = row[0]

                col1_1 = col_array[row_ref] * col_conversion_factor[row_ref]

                new_cols.append(col1_1)

                row_ref = row_ref + 1

            col_ref = col_ref + 1

        new_array = np.array(new_cols).reshape(rows_total, cols_total)

        new_array = np.flip(new_array.swapaxes(0,1), axis=1)

        new_array = np.fliplr(new_array)

        # Update iteration stats
        row_col_matchness = (row_total_sum + col_total_sum) / 2
        iter_stats[iter_count] = row_col_matchness

        if row_col_matchness < closest_match:
            accuracy_improving = False

        if iter_count <= iter_limit:
            print(f"Rebalancing iterations exceeded limit of {iter_limit}")
            break

else:
    new_array

print(f"Stats for the rebalancing\n {iter_stats}")

Rebalancing iterations exceeded limit of 1000
Stats for the rebalancing
 {1: 0.5514141166300177}


In [8]:
df_updated = pd.DataFrame(new_array, columns=cols)
df_updated_column_totals = df_updated.agg('sum').to_frame().T

df_new = pd.concat([df_updated, df_updated_column_totals], axis=0)
df_new = pd.concat([df_new, new_total])
df_new = df_new.reset_index(drop=True).astype('float')

df_new.head()

,A_i,B_i,C_i,Final Demand,0
0,23.799732,71.399195,35.699598,107.098793,NaN
1,70.501553,79.314248,17.625388,96.939636,NaN
2,46.579178,34.934383,11.644794,23.289589,NaN
3,61.227575,122.455150,40.818383,0.000000,NaN
4,202.108038,308.102976,105.788164,227.328018,NaN


In [9]:
new_total_df = new_total.to_frame().reset_index()

new_total_df.columns = ['Industry', 'Total Output']

df_new = pd.concat([df_new, new_total_df], axis=1)[df.columns]

df_new['Industry'].loc[df_new['Industry'] == 'Final Demand'] = 'Value Add'

df_new.head()

,Industry,A_i,B_i,C_i,Final Demand,Total Output
0,A_i,23.799732,71.399195,35.699598,107.098793,220
1,B_i,70.501553,79.314248,17.625388,96.939636,285
2,C_i,46.579178,34.934383,11.644794,23.289589,110
3,Value Add,61.227575,122.455150,40.818383,0.000000,225.0
4,Total Output,202.108038,308.102976,105.788164,227.328018,0.0


### Retain Lists of Column & Row Names for Final DF Output

In [10]:
sector_columns = list(filter(lambda x: (x.endswith('_i')), df_new.columns)) 

sector_columns

['A_i', 'B_i', 'C_i']

In [11]:
industry_columns = df_new.iloc[:,0]

industry_columns.head()

0             A_i
1             B_i
2             C_i
3       Value Add
4    Total Output
Name: Industry, dtype: object

In [12]:
# Create initial Direct Requirements Matrix

df_coefficients = df_new.iloc[:-1, 1:-1]

cols = df_coefficients.columns

coefficients_array = np.array(df_coefficients)

df_coefficients.head()

,A_i,B_i,C_i,Final Demand
0,23.799732,71.399195,35.699598,107.098793
1,70.501553,79.314248,17.625388,96.939636
2,46.579178,34.934383,11.644794,23.289589
3,61.227575,122.455150,40.818383,0.000000
4,202.108038,308.102976,105.788164,227.328018


### Compile Technical Coefficients

In [13]:
df_sectors = df_new[df_new['Industry'].isin(sector_columns)][sector_columns]

df_tc = df_sectors.div(new_total, axis=1)[sector_columns]

tc_matrix = df_tc.to_numpy().astype(float)

df_tc.head()

,A_i,B_i,C_i
0,0.108181,0.250523,0.324542
1,0.320462,0.278296,0.160231
2,0.211724,0.122577,0.105862


### Create Identity Matrix

In [14]:
identity_matrix = np.identity(df_tc.shape[0])

identity_matrix

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

### Create I-A Matrix

In [15]:
i_a_matrix = np.subtract(identity_matrix, tc_matrix)

i_a_matrix

array([[ 0.8918194 , -0.25052349, -0.3245418 ],
       [-0.32046161,  0.72170439, -0.1602308 ],
       [-0.21172353, -0.12257678,  0.89413823]])

### Create I_A Matrix Inverse

In [16]:
i_a_inverse_matrix = np.linalg.inv(i_a_matrix)

i_a_inverse_matrix

array([[1.50632852, 0.63507879, 0.66055306],
       [0.77153439, 1.75438974, 0.59442983],
       [0.46245355, 0.39088875, 1.35629803]])

### Simulate Change in Final Demand

In [17]:
# Note here new_final_demand differs from new_total
# new_total is used for RAS rebalancing to updated the matrix to reflect new total output = sectoral output + final demand
# new_final_demand meanwhile is used to represent change in aggregate demand and how that will impact economy overall due to industry linkages

new_final_demand = np.array([[110],[110],[20]])

new_total_output = np.matmul(i_a_inverse_matrix, new_final_demand)

new_total_output

array([[248.76586495],
       [289.74025163],
       [120.99361401]])

### Update I-O Table to Reflect Increased Demand + Industry Effects

In [18]:
df_new.head()

,Industry,A_i,B_i,C_i,Final Demand,Total Output
0,A_i,23.799732,71.399195,35.699598,107.098793,220
1,B_i,70.501553,79.314248,17.625388,96.939636,285
2,C_i,46.579178,34.934383,11.644794,23.289589,110
3,Value Add,61.227575,122.455150,40.818383,0.000000,225.0
4,Total Output,202.108038,308.102976,105.788164,227.328018,0.0


In [19]:
df_tc = df_new[df_new['Industry'].isin(['Value Add'] + sector_columns)][sector_columns].div(new_total, axis=1)[sector_columns]

tc_array = df_tc.to_numpy().astype(float)

tc_array

array([[0.1081806 , 0.25052349, 0.3245418 ],
       [0.32046161, 0.27829561, 0.1602308 ],
       [0.21172353, 0.12257678, 0.10586177],
       [0.27830716, 0.42966719, 0.37107621]])

In [20]:
updated_sectoral_flows = []

for i, output in enumerate(new_total_output):
    print(f"Sector output: {output[0]}")
    temp_array = []

    for j in tc_matrix[i]:
        temp_array.append(j * output[0])

    updated_sectoral_flows.append(temp_array)

updated_sectoral_flows

Sector output: 248.7658649541721
Sector output: 289.7402516332426
Sector output: 120.99361401313085


[[26.911640285610034, 62.321693292991654, 80.73492085683009],
 [92.85062657754158, 80.63343886997032, 46.42531328877079],
 [25.617195617828227, 14.831007989268976, 12.808597808914113]]

### Rebuild Final DF To Match Initial Structure

In [21]:
df_final = pd.DataFrame(updated_sectoral_flows, columns=sector_columns)
df_new_final_demand = pd.DataFrame(new_final_demand, columns=['Final Demand'])
df_final = pd.concat([df_final, df_new_final_demand], axis=1)

df_updated_column_totals = df_final.agg('sum').to_frame().T
df_final = pd.concat([df_final, df_updated_column_totals], axis=0)

df_final['Total Output'] = df_final.sum(axis=1)
df_final = df_final.reset_index(drop=True).astype('float')

df_final.loc[df_final.index.stop + 1] = df_final.agg('sum', axis=0)

df_final['Industry'] = df_new['Industry']

df_final = df_final[df.columns]

df_final.head()

,Industry,A_i,B_i,C_i,Final Demand,Total Output
0,A_i,26.911640,62.321693,80.734921,110.0,279.968254
1,B_i,92.850627,80.633439,46.425313,110.0,329.909379
2,C_i,25.617196,14.831008,12.808598,20.0,73.256801
3,Value Add,145.379462,157.786140,139.968832,240.0,683.134435
5,NaN,290.758925,315.572280,279.937664,480.0,1366.268869
